In [2]:
from openai import OpenAI
import json
import os

In [3]:
## 定义大模型
client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_BASE_URL"],
)

### 定义工具函数相关
需要定义函数的说明，包括函数的名称，需要的参数，功能描述

In [4]:
# 这里是为了测试，简单定义了两个功能函数

def get_weather(city):
    res = f"现在{city}的天气是晴天，30°"
    return res

def get_current_time():
    res = "现在的时间是2025年8月21日下午16：17"
    return res

# 这里tool的格式参照openai工具调用文档，是固定格式
tools = [{
    "type":"function",
    "function":{
        "name":"get_current_time",
        "description": "获取当前的时间"
    }
},{
    "type":"function",
    "function":{
        "name":"get_weather",
        "description": "获取天气信息",
        "parameters":{
            "type":"object",
            "properties":{
                "city":{
                    "type":"string",
                    "description":"城市名称"
                }
            },
            "required":["city"]
        }
    }
}]

In [5]:
# 方便后续的工具调用
functions = {
    "get_weather": get_weather,
    "get_current_time": get_current_time,
}

### LLM调用
发起一个大模型的流程

In [6]:
system_prompt = "你是一个有用的助手"
prompt = "我想知道当前的具体时间和当前杭州的天气"

messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
res = client.chat.completions.create(
    model = "qwen-max",
    messages=messages,
    tools=tools,
    # parallel_tool_calls=True
)

tool_calls = res.choices[0].message.tool_calls

### 判断
判断大模型返回的内容中是否包含工具调用，如果包含工具调用，需要解析工具，然后手动进行工具调用，然后把工具调用的结果返回给大模型，以进行下一轮的询问迭代。

In [7]:
while tool_calls:
    print("执行工具调用")
    print(f"工具调用名称：{tool_calls[0].function.name}")
    print(f"工具调用参数：{tool_calls[0].function.arguments}")
    # 执行工具函数
    tool_name = tool_calls[0].function.name
    tool_args = tool_calls[0].function.arguments
    if tool_args:
        tool_args = json.loads(tool_args)
        tool_result = functions[tool_name](**tool_args) 
    else:
        tool_result = functions[tool_name]()
    # 注意：模型返回的需要tool调用信息也需要添加到messages中
    messages.append(res.choices[0].message) 
    messages.append({"role": "tool", "content": tool_result, "tool_call_id": tool_calls[0].id})
    res = client.chat.completions.create(
        model = "qwen-max",
        messages=messages,
        tools=tools,
        # parallel_tool_calls=True
    )
    tool_calls = res.choices[0].message.tool_calls

执行工具调用
工具调用名称：get_current_time
工具调用参数：{}
执行工具调用
工具调用名称：get_weather
工具调用参数：{"city": "杭州"}


In [8]:
## 当不需要工具调用时，直接数据结果即可
print(res.choices[0].message.content)

当前的时间是2025年8月21日下午16：17，而杭州现在的天气是晴天，温度为30°。
